# 1장: 기본 프롬프트 구조

- [수업](#lesson)
- [연습문제](#exercises)
- [예제 플레이그라운드](#example-playground)

## 설정

다음 설정 셀을 실행하여 API 키를 로드하고 `get_completion` 헬퍼 함수를 설정합니다.

In [ ]:
%pip install -qU pip
%pip install -qUr requirements.txt

In [ ]:
# Import python's built-in regular expression library
import re
import boto3
from botocore.exceptions import ClientError
import json

# Import the hints module from the utils package
from utils import hints

# Retrieve the MODEL_NAME variable from the IPython store
%store -r modelId
%store -r region

bedrock_client = boto3.client(service_name='bedrock-runtime', region_name=region)

In [ ]:
def get_completion(prompt, system_prompt=None):
    # Define the inference configuration
    inference_config = {
        "temperature": 0.0,  # Set the temperature for generating diverse responses
        "maxTokens": 200,  # Set the maximum number of tokens to generate
        "topP": 1,  # Set the top_p value for nucleus sampling
    }
    # Create the converse method parameters
    converse_api_params = {
        "modelId": modelId,  # Specify the model ID to use
        "messages": [{"role": "user", "content": [{"text": prompt}]}],  # Provide the user's prompt
        "inferenceConfig": inference_config,  # Pass the inference configuration
    }
    # Check if system_text is provided
    if system_prompt:
        # If system_text is provided, add the system parameter to the converse_params dictionary
        converse_api_params["system"] = [{"text": system_prompt}]

    # Send a request to the Bedrock client to generate a response
    try:
        response = bedrock_client.converse(**converse_api_params)

        # Extract the generated text content from the response
        text_content = response['output']['message']['content'][0]['text']

        # Return the generated text content
        return text_content

    except ClientError as err:
        message = err.response['Error']['Message']
        print(f"A client error occured: {message}")

---

## 수업

Amazon Bedrock는 Anthropic Claude 모델에 사용할 수 있는 세 가지 API를 제공합니다. 레거시 [Text Completions API](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-anthropic-claude-text-completion.html), [Messages API](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-anthropic-claude-messages.html) 및 현재 [Converse API](https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference.html)입니다. 이 튜토리얼에서는 Converse API만 사용할 것입니다.

최소한 Converse API를 사용하여 Claude를 호출하려면 다음 매개변수가 필요합니다:
- `modelId`: 호출할 모델의 [API 모델 이름](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids.html#model-ids-arns)

- `messages`: 입력 메시지 배열입니다. Claude 3 모델은 번갈아가는 `user`와 `assistant` 대화 차례로 작동하도록 훈련되었습니다. 새 `Message`를 만들 때 messages 매개변수로 이전 대화 차례를 지정하면 모델이 대화의 다음 `Message`를 생성합니다.
  - 각 입력 메시지는 `role`과 `content`를 가진 객체여야 합니다. 단일 `user`-role 메시지를 지정하거나 여러 `user`와 `assistant` 메시지를 포함할 수 있습니다(번갈아가야 합니다). **첫 번째 메시지는 항상 `user` role을 사용해야 합니다.**
  
  메시지 내용은 [(ContentBlock)](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_ContentBlock.html)의 `content` 필드에 저장합니다. `text` 필드에 텍스트를 지정하거나 모델에서 지원되는 경우 [(ImageBlock)](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_ImageBlock.html)의 `image` 필드에 이미지의 raw bytes를 전달할 수 있습니다. ContentBlock의 다른 필드는 [tool use](https://docs.aws.amazon.com/bedrock/latest/userguide/tool-use.html)를 위한 것입니다.

선택적 매개변수도 있습니다:
- `system`: 시스템 프롬프트 - 아래에서 자세히 설명합니다.
  
- `temperature`: Claude 응답의 가변성 정도입니다. 이 수업과 연습문제에서는 `temperature`를 0으로 설정했습니다.

- `max_tokens`: 생성을 중지하기 전까지 생성할 최대 토큰 수입니다. Claude는 이 최대값에 도달하기 전에 중지할 수 있습니다. 이 매개변수는 생성할 절대 최대 토큰 수만 지정합니다. 또한 이것은 *hard* 중지이므로 Claude가 단어나 문장 중간에 생성을 중지할 수 있습니다.

모든 API 매개변수 목록은 [API 문서](https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference.html)를 방문하세요.

### 예시

Claude가 올바르게 형식화된 프롬프트에 어떻게 응답하는지 살펴보겠습니다. 다음 셀들 중 각각을 실행하면(`shift+enter`), Claude의 응답이 블록 아래에 나타납니다.

In [ ]:
# Prompt
PROMPT = "Hi Claude, how are you?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Can you tell me the color of the ocean?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "What year was Celine Dion born in?"

# Print Claude's response
print(get_completion(PROMPT))

이제 올바른 Converse API 형식을 포함하지 않은 프롬프트를 살펴보겠습니다. 이러한 잘못된 형식의 프롬프트에 대해 Converse API는 오류를 반환합니다.

먼저, `messages` 배열에 `role`과 `content` 필드가 없는 Converse API 호출의 예시를 보겠습니다.

> ⚠️ **경고:** 프롬프트의 messages 매개변수가 잘못된 형식이므로 다음 셀에서 오류가 반환됩니다. 이는 예상된 동작입니다.

In [ ]:
# Get Claude's response
inference_config = {
    "temperature": 0.0, # Set the temperature for generating diverse responses
    "maxTokens": 200 # Set the maximum number of tokens to generate
}

converse_api_params = {
    "modelId": modelId,
    "messages": [{"text":"Hi Claude, how are you?"}], # Provide the user's prompt
    "inferenceConfig": inference_config # Pass the inference configuration
}

response = bedrock_client.converse(**converse_api_params)

# Print Claude's response
print(response['output']['message']['content'][0]['text'])

여기에는 `user`와 `assistant` 역할이 번갈아가지 않는 프롬프트가 있습니다.

> ⚠️ **경고:** `user`와 `assistant` 역할이 번갈아가지 않아 Claude가 오류 메시지를 반환합니다. 이는 예상된 동작입니다.

In [ ]:
inference_config = {
    "temperature": 0.5,
    "maxTokens": 200
}
# Create the converse method parameters
converse_api_params = {
    "modelId": modelId,
    "messages": [
        {"role": "user", "content": [{"text": "What year was Celine Dion born in?"}]},
        {"role": "user", "content": [{"text":"Also, can you tell me some other facts about her?"}]}
    ],
    "inferenceConfig": inference_config,
}

response = bedrock_client.converse(**converse_api_params)

# Print Claude's response
print(response['output']['message']['content'][0]['text'])

`user`와 `assistant` 메시지는 **반드시 번갈아가야 하며**, 메시지는 **`user` 차례로 시작해야 합니다**. 프롬프트에 여러 개의 `user` 및 `assistant` 쌍을 포함하여 다회차 대화를 시뮬레이션할 수 있습니다. 또한 Claude가 이어갈 수 있도록 터미널 `assistant` 메시지에 단어를 넣을 수도 있습니다(이에 대해서는 뒤에서 더 자세히 다룹니다).

#### 시스템 프롬프트

**시스템 프롬프트**도 사용할 수 있습니다. 시스템 프롬프트는 **Claude에게 문맥, 지침 및 가이드라인을 제공하는 방법**입니다. 이는 "User" 차례에서 질문이나 작업을 제시하기 전에 이루어집니다.

구조적으로 시스템 프롬프트는 `user` 및 `assistant` 메시지 목록과 별도로 존재하므로 별도의 `system` 매개변수에 속합니다(노트북의 [설정](#setup) 섹션에서 `get_completion` 헬퍼 함수의 구조를 확인해 보세요).

이 튜토리얼에서 시스템 프롬프트를 사용하는 경우에는 완성 함수에 `system` 필드가 제공됩니다. 시스템 프롬프트를 사용하지 않으려면 `SYSTEM_PROMPT` 변수를 빈 문자열로 설정하면 됩니다.

#### 시스템 프롬프트 예시

In [ ]:
# System prompt
SYSTEM_PROMPT = "Your answer should always be a series of critical thinking questions that further the conversation (do not provide answers to your questions). Do not actually answer the user question."

# Prompt
PROMPT = "Why is the sky blue?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

시스템 프롬프트를 사용하는 이유는 무엇일까요? **잘 작성된 시스템 프롬프트는 Claude의 성능을 다양한 방식으로 향상시킬 수 있습니다.** 예를 들어 Claude가 규칙과 지침을 따르는 능력을 높일 수 있습니다. 자세한 내용은 Anthropic의 [Claude에서 시스템 프롬프트를 사용하는 방법](https://docs.anthropic.com/claude/docs/how-to-use-system-prompts) 문서를 참조하세요.

이제 연습 문제를 풀어보겠습니다. 위의 내용을 변경하지 않고 수업 프롬프트를 실험해보고 싶다면 수업 노트북 맨 아래로 이동하여 [**예시 놀이터**](#example-playground)를 방문하세요.

---

## 연습문제
- [연습문제 1.1 - 세 번 세기](#exercise-11---counting-to-three)
- [연습문제 1.2 - 시스템 프롬프트](#exercise-12---system-prompt)

### 연습 문제 1.1 - 세 번 세기
적절한 `user` / `assistant` 형식을 사용하여 아래의 `PROMPT`를 편집하여 Claude에게 **세 번 세도록** 하세요. 출력 결과에는 귀하의 해답이 정확한지 여부도 표시됩니다.

In [ ]:
# Prompt - this is the only field you should change
PROMPT = "[Replace this text]"

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    pattern = re.compile(r'^(?=.*1)(?=.*2)(?=.*3).*$', re.DOTALL)
    return bool(pattern.match(text))

# Print Claude's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ 힌트가 필요하다면 아래 셀을 실행하세요!

In [ ]:
print(hints.exercise_1_1_hint)

### 연습 문제 1.2 - 시스템 프롬프트

`SYSTEM_PROMPT`를 수정하여 Claude가 3살 아이처럼 응답하도록 만드세요.

In [ ]:
# System prompt - this is the only field you should change
SYSTEM_PROMPT = "[Replace this text]"

# Prompt
PROMPT = "How big is the sky?"

# Get Claude's response
response = get_completion(PROMPT, SYSTEM_PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search(r"giggles", text) or re.search(r"soo", text))

# Print Claude's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ 힌트가 필요하다면 아래 셀을 실행하세요!

In [ ]:
print(hints.exercise_1_2_hint)

### 축하합니다!

지금까지의 모든 연습 문제를 해결했다면 다음 장으로 넘어갈 준비가 되었습니다. 프롬프트 작성을 즐겁게 하세요!

---

## 예제 놀이터

이 영역에서는 이 강의에서 보여준 프롬프트 예제를 자유롭게 실험하고 프롬프트를 수정하여 Claude의 응답이 어떻게 달라지는지 확인할 수 있습니다.

In [ ]:
# Prompt
PROMPT = "Hi Claude, how are you?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Can you tell me the color of the ocean?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "What year was Celine Dion born in?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Get Claude's response
inference_config = {
    "temperature": 0.0
}
additional_model_fields = {
    "max_tokens": 200
}

converse_api_params = {
    "modelId": modelId,
    "messages": [{"text":"Hi Claude, how are you?"}],
    "inferenceConfig": inference_config,
    "additionalModelRequestFields": additional_model_fields
}

response = bedrock_client.converse(**converse_api_params)

# Print Claude's response
print(response['output']['message']['content'][0]['text'])

In [ ]:
inference_config = {
    "temperature": 0.0
}
additional_model_fields = {
    "top_p": 1,
    "max_tokens": 200
}
converse_api_params = {
    "modelId": modelId,
    "messages": [
        {"role": "user", "content": [{"text": "What year was Celine Dion born in?"}]},
        {"role": "user", "content": [{"text":"Also, can you tell me some other facts about her?"}]}
    ],
    "inferenceConfig": inference_config,
    "additionalModelRequestFields": additional_model_fields
}

response = bedrock_client.converse(**converse_api_params)

# Print Claude's response
print(response['output']['message']['content'][0]['text'])

In [ ]:
# System prompt
SYSTEM_PROMPT = "Your answer should always be a series of critical thinking questions that further the conversation (do not provide answers to your questions). Do not actually answer the user question."

# Prompt
PROMPT = "Why is the sky blue?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))